In [0]:
import pandas as pd
import numpy as np
import requests
import bs4
url = 'https://www.monsterindia.com/data-science-jobs.html'

page = requests.get(url).text
bs_obj = bs4.BeautifulSoup(page, 'html.parser')

In [0]:
jobs = pd.DataFrame()
for i in range(1,5):
  url = 'https://www.monsterindia.com/data-science-jobs-%d.html' % i
  page = requests.get(url).text
  bs_obj = bs4.BeautifulSoup(page, 'html.parser')
  
  parents = bs_obj.findAll('div', {'class': 'jobwrap',
                                   'type': 'tuple'})
  for parent in parents:
    job_title = parent.find('span', {'class': 'title_in'}).text
    company = parent.find('div', {'class': 'jtxt orange'}).text

    keyskills1 = parent.find('div', {'class': 'joblnk serachjoblnk'})
    keyskills2 = keyskills1.findAll('div', {'class': 'jtxt'})[3]
    keyskills = keyskills2.findAll('span')[1].text

    location_tag = parent.find('div', {'class': 'jtxt jico ico1'})
    if location_tag is None:
      location = None
    else:
      location = location_tag.text

    summary1 = parent.find('div', {'class': 'joblnk serachjoblnk'})
    try:
      summary2 = summary1.findAll('div', {'class': 'jtxt'})[4]
      summary = summary2.findAll('span')[1].text
    except:
      summary = None

    job_link = parent.find('a', {'class': 'title_in'}).get('href')
    job_id = parent.get('id')
    curr_row = {'job_id': job_id,
                'job_title': job_title,
                'company': company,
                'keyskills': keyskills,
                'location': location,
                'summary': summary,
                'job_link': job_link}
    jobs = jobs.append(curr_row, ignore_index=True)

In [74]:
jobs_detailed = pd.DataFrame()
for i, row in jobs.iterrows():
  url = row['job_link'].replace(' ', '%20')
  page = requests.get('https:'+url).text
  bs_obj = bs4.BeautifulSoup(page, 'html.parser')
  
  role_sec = bs_obj.find('div', {'class': 'jd_rol_section'})
  span_tags = role_sec.findAll('span')
  
  industry = span_tags[0].text
  function = span_tags[1].text
  #roles = span_tags[2].text
  roles = '--'.join([a.text for a in span_tags[2].findAll('a')])
  posted_on = span_tags[3].text
  
  curr_row = {'job_id': row['job_id'],
              'industry': industry,
              'function': function,
              'roles': roles,
              'posted_on': posted_on}
  jobs_detailed = jobs_detailed.append(curr_row, ignore_index=True)
jobs_detailed

AttributeError: ignored